In [4]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import copy
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from ipywidgets import interact
from torchsummary import summary
import covid_19_deep_learning as c19

In [5]:
# covid_19_deep_learning.py에 저장된 딥러닝 모델 객체 불러오기

device = torch.device('cpu')

dataloaders = c19.covid_dataloader(c19.train_data_dir, c19.val_data_dir)
model = c19.build_covid_model(device_name='cpu')

# CrossEntropyLoss : 다중 분류에 사용하는 손실함수. nn.LogSoftmax(신경망 말단의 결과 값들을 확률개념으로 해석하기 위한 Softmax 함수의 결과에 log 값을 취한 연산)와 
# nn.NLLLoss(nn.LogSoftmax의 log 결과값에 대한 교차 엔트로피 손실 연산(Cross Entropy Loss|Error) 연산의 조합. 
loss_func = nn.CrossEntropyLoss(reduction='mean')
# 확률적 경사 하강법(SGD : Stochastic Gradient Descent) : 경사하강법처럼 한번 학습할 때 모든 데이터에 대해 가중치를 조절하지 않고,
# 랜덤하게 추출한 일부 데이터에 대해 가중치를 조절. 최적값을 얻기는 힘들지만 속도는 훨씬 빠름
# momentum : 최적화를 진행할 때, 이전 step에서의 update vector도 고려하는 방법
optimizer = torch.optim.SGD(model.parameters(), lr=1E-3, momentum=0.9)

summary(model, (3, 224, 224), batch_size=1, device='cpu')

c:\Users\user\AppData\Local\Programs\Python\Python37\lib\site-packages\torchvision\models\_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
c:\Users\user\AppData\Local\Programs\Python\Python37\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [1, 64, 224, 224]           1,792
              ReLU-2          [1, 64, 224, 224]               0
            Conv2d-3          [1, 64, 224, 224]          36,928
              ReLU-4          [1, 64, 224, 224]               0
         MaxPool2d-5          [1, 64, 112, 112]               0
            Conv2d-6         [1, 128, 112, 112]          73,856
              ReLU-7         [1, 128, 112, 112]               0
            Conv2d-8         [1, 128, 112, 112]         147,584
              ReLU-9         [1, 128, 112, 112]               0
        MaxPool2d-10           [1, 128, 56, 56]               0
           Conv2d-11           [1, 256, 56, 56]         295,168
             ReLU-12           [1, 256, 56, 56]               0
           Conv2d-13           [1, 256, 56, 56]         590,080
             ReLU-14           [1, 256,

In [6]:
# 불러온 딥러닝 모델 객체로 정상 / covid-19 / 바이러스성 폐렴의 이미지 데이터 학습

num_epochs = 10

best_acc = 0.0
train_loss, train_accuracy = [], []
val_loss, val_accuracy = [], []

for epoch in range(num_epochs):
    # 학습한 epoch의 loss 및 accuracy를 각 train 및 val 리스트에 추가
    losses, accuracies = c19.train_covid(dataloaders, model, optimizer, loss_func, device)
    train_loss.append(losses['train'])
    val_loss.append(losses['val'])
    train_accuracy.append(accuracies['train'])
    val_accuracy.append(accuracies['val'])
    
    print(f"{epoch+1}/{num_epochs}-Train Loss:{losses['train']}, Val Loss:{losses['val']}")
    print(f"{epoch+1}/{num_epochs}-Train Acc:{accuracies['train']}, Val Acc:{accuracies['val']}")
    
    # deepcopy: 배열의 내부 객체까지 복사를 해서 사용
    # copy: 배열의 내부 객체까지 깊은 복사를 해주지 않음
    # epoch을 4번 이상 진행한 상태에서 최적의 정확도가 나왔다면 해당 모델을 pth파일로 저장
    if (epoch > 3) and (accuracies['val'] > best_acc):
        best_acc = accuracies['val']
        best_model = copy.deepcopy(model.state_dict())
        c19.save_best_model(best_model, f'model_{epoch+1:02d}.pth')

print(f'Bset Accuracy: {best_acc}')

0/62 - Running loss: 1.1119983196258545


KeyboardInterrupt: 

In [19]:
# cv2로 이미지 시각화
def get_RGB_image(data_dir, file_name):
    image_file = os.path.join(data_dir, file_name)
    image = cv2.imread(image_file)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

# 이미지를 검증에 사용할 텐서 형태로 변환
def preprocess_image(image):
    transformer = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((224, 224)),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])

    tensor_image = transformer(image) # (C, H, W)
    tensor_image = tensor_image.unsqueeze(0) # (B, C, H, W)
    return tensor_image

# 모델을 통해 검증할 이미지를 텐서로 변환 후 예측
def model_predict(image, model):
    tensor_image = preprocess_image(image)
    prediction = model(tensor_image)

    _, pred_label = torch.max(prediction.detach(), dim=1)
    pred_label = pred_label.squeeze(0)
    # 모델을 통해 예측된 label을 반환
    return pred_label.item()

In [20]:
data_dir = './Covid19-dataset/train/'

class_list = ['Normal', 'Covid', 'Viral Pneumonia']
test_normals_list = c19.list_image_file(data_dir, 'Normal')
test_covids_list = c19.list_image_file(data_dir, 'Covid')
test_pneumonias_list = c19.list_image_file(data_dir, 'Viral Pneumonia')

In [21]:
# 학습된 모델 데이터 불러오기(.pth 파일)
loaded_model = torch.load('./trained_model/model_10.pth')

model = c19.build_covid_model(device_name='cpu')
model.load_state_dict(loaded_model)


c:\Users\user\AppData\Local\Programs\Python\Python37\lib\site-packages\torchvision\models\_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
c:\Users\user\AppData\Local\Programs\Python\Python37\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


<All keys matched successfully>

In [22]:
min_num_files = min(len(test_normals_list), len(test_covids_list), len(test_pneumonias_list))

# 모델 시각화
@interact(index=(0, min_num_files-1))
def show_result(index=0):
    normal_image = get_RGB_image(data_dir, test_normals_list[index])
    covid_image = get_RGB_image(data_dir, test_covids_list[index])
    pneumonia_image = get_RGB_image(data_dir, test_pneumonias_list[index])
    
    prediction_1 = model_predict(normal_image, model)
    prediction_2 = model_predict(covid_image, model)
    prediction_3 = model_predict(pneumonia_image, model)
    
    plt.figure(figsize=(12, 8))
    plt.subplot(131)
    plt.title(f'Pred:{class_list[prediction_1]} | GT:Normal')
    plt.imshow(normal_image)
    plt.subplot(132)
    plt.title(f'Pred:{class_list[prediction_2]} | GT:Covid')
    plt.imshow(covid_image)
    plt.subplot(133)
    plt.title(f'Pred:{class_list[prediction_3]} | GT:Pneumonia')
    plt.imshow(pneumonia_image)
    plt.tight_layout()

interactive(children=(IntSlider(value=0, description='index', max=69), Output()), _dom_classes=('widget-intera…